In [49]:
import numpy as np
import random
import time

def generate_tenner_grid():
    rows, cols = 3, 10
    grid = -np.ones((rows, cols), dtype=int)  # Initialize with -1 which means empty

    # Function to check if placement is valid
    def valid_placement(grid, r, c, num):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, -1), (-1, 1), (1, 1)]
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == num:
                    return False
        return True

    # Attempt to place numbers 0-9 in each row with all conditions satisfied
    for r in range(rows):
        available_numbers = list(range(cols))
        for c in range(cols):
            valid_numbers = [num for num in available_numbers if valid_placement(grid, r, c, num)]
            if not valid_numbers:
                return generate_tenner_grid()  # Restart grid generation if stuck
            choice = random.choice(valid_numbers)
            grid[r][c] = choice
            available_numbers.remove(choice)

    # Print the full grid before deleting cells
    print("Full grid before deleting cells:")
    print(grid)

    # Calculate column sums
    column_sums = grid.sum(axis=0)

    # Randomly empty some cells
    num_empty_cells = random.randint(5, 15)  # Randomly empty between 5 and 15 cells
    empty_positions = set()
    while len(empty_positions) < num_empty_cells:
        empty_r = random.randint(0, rows - 1)
        empty_c = random.randint(0, cols - 1)
        empty_positions.add((empty_r, empty_c))
    for r, c in empty_positions:
        grid[r][c] = -1

    # Print the grid after randomly deleting cells
    print("Grid after randomly deleting cells:")
    print(grid)
    print('-----------------------------------------------')

    return grid, column_sums.tolist()

class Cell:
    def __init__(self, initial_value):
        self.value = initial_value
        if initial_value == -1:
            self.domain = list(range(10))  # Values from 0 to 9
        else:
            self.domain = [initial_value]  # The domain is the initial value if it's not -1

#Simple Backtracking

class TennerGridB:
    def __init__(self, initial_grid, column_sums):
        self.rows = len(initial_grid)
        self.cols = len(initial_grid[0])
        self.grid = [[Cell(initial_grid[r][c]) for c in range(self.cols)] for r in range(self.rows)]
        self.column_sums = column_sums
        self.consistency_checks = 0
        self.assignment_count = 0

    def is_safe(self, row, col, num):
        # Check row uniqueness
        for i in range(self.cols):
            if i != col and self.grid[row][i].value == num:
                self.consistency_checks += 1
                return False

        # Check column sum
        current_sum = sum(self.grid[r][col].value for r in range(self.rows) if self.grid[r][col].value != -1)
        if current_sum + num > self.column_sums[col]:
            self.consistency_checks += 1
            return False

        # Check adjacent cells
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (-1, -1), (1, 1), (1, -1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr][nc].value == num:
                self.consistency_checks += 1
                return False

        return True

    def solve(self, row=0, col=0):
        if col == self.cols:
            col = 0
            row += 1
        if row == self.rows:
            return True

        cell = self.grid[row][col]
        if cell.value != -1:
            return self.solve(row, col + 1)

        for value in cell.domain[:]:  # Work on a copy to avoid modifying the original while iterating
            if self.is_safe(row, col, value):
                cell.value = value
                self.assignment_count += 1
                if self.solve(row, col + 1):
                    return True
                cell.value = -1  # Backtrack

        return False

    def print_state(self):
        print('Grid:')
        for row in self.grid:
            print(' '.join(str(cell.value) if cell.value != -1 else '.' for cell in row))
        print("Column sums:", self.column_sums)
        print('Assignments:', self.assignment_count)
        print('Consistency checks:', self.consistency_checks)

#Forward checking
class TennerGridFC:
    def __init__(self, initial_grid, column_sums):
        self.rows = len(initial_grid)
        self.cols = len(initial_grid[0])
        self.grid = [[Cell(initial_grid[r][c]) for c in range(self.cols)] for r in range(self.rows)]
        self.column_sums = column_sums
        self.consistency_checks = 0
        self.assignment_count = 0

    def is_safe(self, row, col, num):
        # Check row uniqueness
        for i in range(self.cols):
            if i != col and self.grid[row][i].value == num:
                self.consistency_checks += 1
                return False

        # Check column sum
        current_sum = sum(self.grid[r][col].value for r in range(self.rows) if self.grid[r][col].value != -1)
        if current_sum + num > self.column_sums[col]:
            self.consistency_checks += 1
            return False

        # Check adjacent cells
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (-1, -1), (1, 1), (1, -1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr][nc].value == num:
                self.consistency_checks += 1
                return False

        return True

    def forward_check(self, row, col, value):
        removals = []
        # Remove the assigned value from the domains of all unassigned cells in the row and column
        for r in range(self.rows):
            if r != row and value in self.grid[r][col].domain:
                self.grid[r][col].domain.remove(value)
                removals.append((r, col, value))
        for c in range(self.cols):
            if c != col and value in self.grid[row][c].domain:
                self.grid[row][c].domain.remove(value)
                removals.append((row, c, value))
        return removals

    def restore_domains(self, removals):
        for r, c, value in removals:
            if value not in self.grid[r][c].domain:
                self.grid[r][c].domain.append(value)

    def solve(self, row=0, col=0):
        if col == self.cols:
            col = 0
            row += 1
        if row == self.rows:
            return True

        cell = self.grid[row][col]
        if cell.value != -1:
            return self.solve(row, col + 1)

        original_domain = cell.domain[:]
        for value in original_domain:
            if self.is_safe(row, col, value):
                cell.value = value
                self.assignment_count += 1
                removals = self.forward_check(row, col, value)
                if self.solve(row, col + 1):
                    return True
                # Backtrack
                cell.value = -1
                self.restore_domains(removals)

        return False

    def print_state(self):
        print('Grid:')
        for row in self.grid:
            print(' '.join(str(cell.value) if cell.value != -1 else '.' for cell in row))
        print("Column sums:", self.column_sums)
        print('Assignments:', self.assignment_count)
        print('Consistency checks:', self.consistency_checks)


#Forward checking with MRV
class TennerGridMRV:
    def __init__(self, input_grid, column_sums):
        self.rows = len(input_grid)
        self.cols = len(input_grid[0])
        self.grid = [[Cell(input_grid[r][c]) for c in range(self.cols)] for r in range(self.rows)]
        self.column_sums = column_sums
        self.consistency_checks = 0
        self.assignment_count = 0

    def is_safe(self, row, col, num):
        # Check row uniqueness
        for i in range(self.cols):
            if i != col and self.grid[row][i].value == num:
                self.consistency_checks += 1
                return False

        # Check column sum
        current_sum = sum(self.grid[r][col].value for r in range(self.rows) if self.grid[r][col].value != -1)
        if current_sum + num > self.column_sums[col]:
            self.consistency_checks += 1
            return False

        # Check adjacent cells
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (-1, -1), (1, 1), (1, -1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr][nc].value == num:
                self.consistency_checks += 1
                return False

        return True

    def forward_check(self, row, col, value):
        removals = []
        # Remove the assigned value from the domains of all unassigned cells in the same row, column, and adjacent cells
        for r in range(self.rows):
            for c in range(self.cols):
                if (r != row or c != col) and (r == row or c == col or ((abs(r - row) == 1 or abs(r - row) == 0) and (abs(c - col) == 1 or abs(c - col) == 0))):
                    if value in self.grid[r][c].domain and self.grid[r][c].value == -1:
                        self.grid[r][c].domain.remove(value)
                        removals.append((r, c, value))
        return removals

    def restore_domains(self, removals):
        for (r, c, val) in removals:
            if val not in self.grid[r][c].domain:  
                self.grid[r][c].domain.append(val)

    def find_mrv_cell(self):
        min_domain_size = float('inf')
        min_cell = None
        for r in range(self.rows):
            for c in range(self.cols):
                if self.grid[r][c].value == -1 and len(self.grid[r][c].domain) < min_domain_size:
                    min_domain_size = len(self.grid[r][c].domain)
                    min_cell = (r, c)
        return min_cell

    def solve(self):
        cell = self.find_mrv_cell()
        if cell is None:  # Check if no unassigned cells are left
            return True

        row, col = cell
        for num in self.grid[row][col].domain[:]:
            if self.is_safe(row, col, num):
                self.grid[row][col].value = num
                self.assignment_count += 1
                removals = self.forward_check(row, col, num)

                if self.solve():
                    return True

                # Backtrack and restore domains
                self.grid[row][col].value = -1
                self.restore_domains(removals)

        return False

    def print_state(self):
        print('Grid:')
        for row in self.grid:
            print(' '.join(str(cell.value) if cell.value != -1 else '.' for cell in row))
        print("Column sums:", self.column_sums)
        print('Assignments:', self.assignment_count)
        print('Consistency checks:', self.consistency_checks)

def main():
    initial_grid, column_sums = generate_tenner_grid()
    grid = TennerGridB(initial_grid, column_sums)
    print('Initial State:')
    grid.print_state()
    print('-----------------------------------------------')
    print('Simple Backtracking')
    start_time = time.perf_counter()
    sol= grid.solve()
    finish_time = time.perf_counter() 
    if sol: 
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')
    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')
    
    print('Forward Checking')
    grid = TennerGridFC(initial_grid, column_sums)
    start_time = time.perf_counter()
    sol= grid.solve()
    finish_time = time.perf_counter()
    if sol: 
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')
    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')

    print('Forward Checking with MRV')
    grid = TennerGridMRV(initial_grid, column_sums)
    start_time = time.perf_counter()
    sol= grid.solve()
    finish_time = time.perf_counter()
    if sol: 
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')
    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')
if __name__ == "__main__":
    main()

Full grid before deleting cells:
[[1 9 8 7 5 6 2 4 3 0]
 [5 6 3 1 0 7 8 9 2 4]
 [7 1 0 5 9 2 3 4 8 6]]
Grid after randomly deleting cells:
[[-1  9  8  7  5  6  2  4 -1  0]
 [ 5 -1 -1  1 -1  7  8  9  2 -1]
 [ 7  1  0 -1  9  2  3  4  8  6]]
-----------------------------------------------
Initial State:
Grid:
. 9 8 7 5 6 2 4 . 0
5 . . 1 . 7 8 9 2 .
7 1 0 . 9 2 3 4 8 6
Column sums: [13, 16, 11, 13, 14, 15, 13, 17, 13, 10]
Assignments: 0
Consistency checks: 0
-----------------------------------------------
Simple Backtracking
Solved State:
Grid:
1 9 8 7 5 6 2 4 3 0
5 6 3 1 0 7 8 9 2 4
7 1 0 5 9 2 3 4 8 6
Column sums: [13, 16, 11, 13, 14, 15, 13, 17, 13, 10]
Assignments: 11
Consistency checks: 58
Elapsed time: 0.000582 seconds
-----------------------------------------------
Forward Checking
Solved State:
Grid:
1 9 8 7 5 6 2 4 3 0
5 6 3 1 0 7 8 9 2 4
7 1 0 5 9 2 3 4 8 6
Column sums: [13, 16, 11, 13, 14, 15, 13, 17, 13, 10]
Assignments: 11
Consistency checks: 55
Elapsed time: 0.000691 seconds
